<span style="color: #00857e; font-family: Segoe UI; font-size: 1.8em; font-weight: 300;">EfficintNetB7 2 Class Image Level</span>

In [1]:
!pip install efficientnet -q

In [2]:
!pip install wandb --upgrade

     |████████████████████████████████| 1.8 MB 2.8 MB/s 
  Attempting uninstall: wandb
    Found existing installation: wandb 0.10.33
    Uninstalling wandb-0.10.33:
      Successfully uninstalled wandb-0.10.33
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.5.0 requires wandb<0.11.0,>=0.10.0, but you have wandb 0.11.0 which is incompatible.


In [3]:
import tensorflow_addons as tfa

In [4]:
import os

import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import GroupKFold

import csv
from kaggle_datasets import KaggleDatasets

from tensorflow.keras import layers

In [5]:
import wandb
from wandb.keras import WandbCallback

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")

wandb.login(key=secret_value_0)

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

In [9]:
import random

In [10]:
def build_augmenter(with_labels=True):
    def augment(img):
        #print("img.shape", img.shape)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        
        #img = tf.image.random_saturation(img, 0.5, 1.0)
        #img = tf.image.random_saturation(img, 0.2, 0.5)
        #img = tf.image.random_brightness(img, 0.2)
    
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) 
    
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize)
    
    #if augment:
    #    dset = dset.map(lambda x, y: (data_augmentation(x, training=augment), y), 
    #                num_parallel_calls=AUTO)
    
    dset = dset.prefetch(AUTO)
    
    return dset

In [12]:
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 8

Running on TPU: grpc://10.0.0.2:8470
Running on 8 replicas


In [13]:
COMPETITION_NAME = "covid19512"
GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)

In [14]:
_data_dir = GCS_DS_PATH
load_dir = _data_dir + "/train/"
df = pd.read_csv(_data_dir + '/updated_iamge_level.csv')

In [15]:
df.head()

,Unnamed: 0,id,boxes,label,StudyInstanceUID,bin,fold
0,0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,1.0,2.0
1,1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,0.0,4.0
2,2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,1.0,4.0
3,3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,3.0,2.0
4,4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,1.0,2.0


In [16]:
df['none'] = 0 
for i, r in df.iterrows():
    if r.label == "none 1 0 0 1 1":
        df.loc[i, 'none'] = 1
#df

In [18]:
df['id'] = df['id'].str.replace("_image", "")

In [19]:
IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)
IMS = 8

In [20]:
for i in range(5):
    
    config={
        "fold": i,
        "start_lr": 1e-3,
        "min_lr":1e-6,
        "epoch": 30,
        "batch_size": BATCH_SIZE,
        "IMG_SIZE": IMSIZE[IMS]
    }
    wandb.init(project="covid19-siim-2class-enb7", config=config)
    
    valid_paths = load_dir + df[df['fold'] == i]['id'] + '.png' #"/train/"
    train_paths = load_dir + df[df['fold'] != i]['id'] + '.png' #"/train/" 
    valid_labels = df[df['fold'] == i]['none'].values
    train_labels = df[df['fold'] != i]['none'].values

    

    decoder = build_decoder(with_labels=True, target_size=(IMSIZE[IMS], IMSIZE[IMS]), ext='png')
    test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[IMS], IMSIZE[IMS]),ext='png')

    train_dataset = build_dataset(
        train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder
        # sequence augment
        #, augment=False
    )

    valid_dataset = build_dataset(
        valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
        repeat=False, shuffle=False, augment=False
    )

    try:
        n_labels = train_labels.shape[1]
    except:
        n_labels = 1

    with strategy.scope():
        model = tf.keras.Sequential([
            
            efn.EfficientNetB7(
                input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
                weights='noisy-student',
                include_top=False),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(n_labels, activation='sigmoid')
        ])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            #optimizer = tfa.optimizers.RectifiedAdam(learning_rate=config['start_lr']),
            #optimizer=tfa.optimizers.Lookahead(
            #    #tf.keras.optimizers.Adam(learning_rate=config['start_lr']),
            #    tfa.optimizers.RectifiedAdam(learning_rate=config['start_lr']),
            #    sync_period=6,
            #    slow_step_size=0.5
            #),
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.AUC(multi_label=True)])
            #metrics=[tf.keras.metrics.AUC()])

        model.summary()


    steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        f'model{i}.h5', save_best_only=True, monitor='val_loss', mode='min')
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", patience=3, min_lr=config["min_lr"], mode='min')

    history = model.fit(
        train_dataset, 
        epochs=config["epoch"],
        verbose=1,
        callbacks=[checkpoint, lr_reducer, WandbCallback()],
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_dataset)

    hist_df = pd.DataFrame(history.history)
    hist_df.to_csv(f'history{i}.csv')
    wandb.finish()

wandb: Currently logged in as: liortoledano (use `wandb login --relogin` to force relogin)


258072576/258068648 [==============================] - 8s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Functional) (None, 16, 16, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2561      
Total params: 64,100,241
Trainable params: 63,789,521
Non-trainable params: 310,720
_________________________________________________________________
Epoch 1/30
77/77 [==============================] - 415s 2s/step - loss: 0.6326 - auc: 0.6190 - val_loss: 0.5706 - val_auc: 0.8011
Epoch 2/30
77/77 [====================

epoch,29
loss,0.30183
auc,0.93257
val_loss,0.41477
val_auc,0.87789
lr,0.0
_runtime,1826
_timestamp,1627425665
_step,29
best_val_loss,0.39844
best_epoch,13


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▆▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▁▂▁▂▁▂▁▁▁▁▁▁▁
auc,▁▃▄▄▅▅▆▆▆▇▇▇▇▇████████████████
val_loss,▂█▂▅▂▁▁▂▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc,▂▃▆▅▅▇▆▆▁▇██████▇▇█▇██▇▇██▇█▇█
lr,█████████▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Functional) (None, 16, 16, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2560)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2560)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2561      
Total params: 64,100,241
Trainable params: 63,789,521
Non-trainable params: 310,720
_________________________________________________________________
Epoch 1/30
77/77 [==============================] - 352s 2s/step - loss: 0.6119 - auc_1: 0.6367 - val_loss: 0.7637 - val_auc_1: 0.7226
Epoch 2/30
77/77 [==============================] - 39s 512ms/step - loss: 0.5146 - auc_1: 0.7659 -

...

epoch,29
loss,0.34596
auc_3,0.90895
val_loss,0.39015
val_auc_3,0.89231
lr,0.0
_runtime,1713
_timestamp,1627430919
_step,29
best_val_loss,0.37572
best_epoch,8


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▆▆▅▅▄▃▃▂▃▂▂▂▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁
auc_3,▁▃▄▅▅▆▆▇▇▇▇████▇██████████████
val_loss,▄▂▄▄█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc_3,▁▂▅▅▅▇▇████▇██████▇▇▇▇▇▇█▇▇▇▇█
lr,█████▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
_timestamp,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Functional) (None, 16, 16, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2560)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 2560)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 2561      
Total params: 64,100,241
Trainable params: 63,789,521
Non-trainable params: 310,720
_________________________________________________________________
Epoch 1/30
77/77 [==============================] - 375s 2s/step - loss: 0.6159 - auc_4: 0.6614 - val_loss: 1.4806 - val_auc_4: 0.7516
Epoch 2/30
77/77 [==============================] - 39s 512ms/step - loss: 0.5274 - auc_4: 0.7730 -

epoch,29
loss,0.27764
auc_4,0.94195
val_loss,0.44612
val_auc_4,0.87382
lr,0.0
_runtime,1802
_timestamp,1627432747
_step,29
best_val_loss,0.38764
best_epoch,13


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▇▆▆▅▅▅▅▄▄▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
auc_4,▁▃▄▄▅▅▅▅▆▆▇▇▇▇▇███████████████
val_loss,█▃▂▆▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc_4,▁▅▅▃▆▇▇▆▇████████▇██▇█▇█▇▇█▇█▇
lr,█████████▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
